This project is "Abstractive Text Summarization using Transformers (T5)".

Unlike basic extractive summarization (which just picks important sentences), Abstractive Summarization uses Deep Learning to understand the context and write a new summary in its own words, similar to how a human would.

We will use T5 (Text-To-Text Transfer Transformer), a state-of-the-art model by Google, which is capable of summarizing, translating, and more.

Cell 1: Install Necessary Libraries
We need the Hugging Face transformers library to download the pre-trained T5 model and sentencepiece for text processing.


In [7]:
!pip install -q transformers sentencepiece safetensors
# We also need PyTorch or TensorFlow. Since you used TF before, we'll install it just in case,
# but Hugging Face works seamlessly with both.
!pip install -q tensorflow

Cell 2: Import and Load the Pre-trained Model
We will load the T5-small model. It's lightweight and runs fast on free GPUs (like Colab), yet it produces surprisingly good summaries. We use the TF version (TFAutoModel...) to stay consistent with your TensorFlow environment.

In [9]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# 1. Load the Tokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 2. Load the Model (The Deep Learning Network)
# FIX: We add `use_safetensors=False` to force it to look for the native
# TensorFlow weights (.h5 file) instead of trying to convert the PyTorch weights.
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint, use_safetensors=False)

print(f"✅ Model '{checkpoint}' loaded successfully.")

tf_model.h5:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model 't5-small' loaded successfully.


Cell 3: Define the Summarization Function
T5 is unique because it treats every NLP problem as a "text-to-text" problem. To tell it we want to summarize, we must add the prefix "summarize: " to our input text.

In [10]:
def summarize_text(text, max_length=150, min_length=40):
    """
    Summarizes the input text using the loaded T5 model.
    """
    # 1. Preprocess: Add the specific prefix for T5
    prefix = "summarize: "
    inputs = prefix + text

    # 2. Tokenize: Convert to tensors
    # 'pt' = PyTorch, 'tf' = TensorFlow. We use 'tf' here.
    tokenized_inputs = tokenizer(inputs, return_tensors="tf", max_length=1024, truncation=True)

    # 3. Generate Summary IDs
    # num_beams=4 ensures the model explores multiple sentence possibilities (Beam Search)
    summary_ids = model.generate(
        tokenized_inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # 4. Decode: Convert IDs back to human-readable text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

print("✅ Summarization function ready.")

✅ Summarization function ready.


Cell 4: Test with a Long Article
Run this cell to see the AI read a long text about SpaceX and generate a concise summary.

In [11]:
# A sample long text (Abstractive models shine on long, complex text)
text_to_summarize = """
SpaceX, founded by Elon Musk in 2002, is a private aerospace manufacturer and space transportation services company headquartered in Hawthorne, California.
It was founded with the goal of reducing space transportation costs to enable the colonization of Mars.
SpaceX has developed several launch vehicles, the Starlink satellite constellation, and the Dragon cargo spacecraft.
In 2008, SpaceX's Falcon 1 became the first privately developed liquid-propellant launch vehicle to reach orbit.
In 2010, the Dragon spacecraft became the first commercially built and operated spacecraft to be recovered successfully from orbit.
SpaceX's achievements include the first vertical take-off and vertical propulsive landing for an orbital rocket, the first reuse of an orbital rocket,
and the first private company to send astronauts to orbit and to the International Space Station (ISS).
Musk has stated that the long-term goal of the company is the colonization of Mars, with the Starship rocket being developed for that purpose.
"""

print("📝 ORIGINAL TEXT:")
print(text_to_summarize)
print("-" * 50)

# Generate Summary
summary = summarize_text(text_to_summarize)

print("\n🤖 AI GENERATED SUMMARY:")
print(summary)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


📝 ORIGINAL TEXT:

SpaceX, founded by Elon Musk in 2002, is a private aerospace manufacturer and space transportation services company headquartered in Hawthorne, California. 
It was founded with the goal of reducing space transportation costs to enable the colonization of Mars. 
SpaceX has developed several launch vehicles, the Starlink satellite constellation, and the Dragon cargo spacecraft. 
In 2008, SpaceX's Falcon 1 became the first privately developed liquid-propellant launch vehicle to reach orbit. 
In 2010, the Dragon spacecraft became the first commercially built and operated spacecraft to be recovered successfully from orbit. 
SpaceX's achievements include the first vertical take-off and vertical propulsive landing for an orbital rocket, the first reuse of an orbital rocket, 
and the first private company to send astronauts to orbit and to the International Space Station (ISS). 
Musk has stated that the long-term goal of the company is the colonization of Mars, with the Stars

Cell 5 (Optional): Interactive Mode
Run this cell to paste your own text (e.g., from a news article or Wikipedia) and get a summary instantly.

In [12]:
# Simple loop to try your own text
while True:
    user_input = input("\nPaste text to summarize (or type 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break

    if len(user_input) < 50:
        print("⚠️ Text too short. Please enter a longer paragraph.")
        continue

    print("\nProcessing...")
    result = summarize_text(user_input)
    print(f"\n📢 Summary: {result}")


Paste text to summarize (or type 'quit' to exit): Dhurandhar (transl. Stalwart)[4] is a 2025 Indian Hindi-language spy action thriller film written, co-produced, and directed by Aditya Dhar. Produced by Jyoti Deshpande, Aditya Dhar, and Lokesh Dhar under Jio Studios and B62 Studios, it stars Ranveer Singh, Akshaye Khanna, Sanjay Dutt, R. Madhavan, Arjun Rampal, and several other actors.[5] The first instalment of a two-part film series, it depicts an anti-terror covert operation,[4] where an undercover agent infiltrates Karachi's criminal and political underworld in Pakistan. The plot loosely joins several real-life events involving geopolitical tensions such as the 1999 IC-814 hijacking, the 2001 Indian Parliament attack, the 2008 Mumbai attacks, and events surrounding Operation Lyari.[6][7]

Processing...

📢 Summary: the film is a 2025 Indian Hindi-language spy action thriller. it stars Ranveer Singh, Akshaye Khanna, Sanjay Dutt, R. Madhavan, Arjun Rampal. it is the first instalment